# Atividade 2 - Matemática Computacional


Configurações Iniciais do Notebook

In [42]:
# %pip install numpy matplotlib ipympl
# %matplotlib notebook

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Questão A

### Definição do Dataset
Utilizaremos os seguintes pontos hipotéticos no gráfico.
$$ Dataset = \{(0.75,1.5),(2.5,2.0),(4.0,4.0),(5,4.5)\} $$

Em código, os dataset acima fica definido:

In [60]:
# 1. Configuração dos Dados e Parâmetros
X = np.array([0.75, 2.5, 4.0 , 5])   # Peso
y = np.array([1.5, 2.0, 4.0 , 4.5])    # Altura

Para o exemplo A do slide, o valor do slope já é conhecido, que nesse caso equivale a 0.64, nosso foco nesse exemplo se dará apenas na otimização do intercepto.
Utilizaremos os valores definidos abaixo em código.

In [61]:
slope_fixed = 0.64 # Valor fornecido pelo comando da atividade
b_start = -0.5 # Valor inicial do intercepto (b)
learning_rate = 0.05 # Taxa de aprendizado (α)
epochs = 15 # Número de iterações

A função calculate_ssr(), avalia o valor real e o previsto para a função fornecida, e calcula o resíduo quadrado. Essa função obedece a fórmula:
$$ SSR = \sum^n_{i=1}(y_i - \hat{y}_i)^2 $$
onde o $ y_i $ é o valor real e $ \hat{y}_i $ é o valor previsto pelo modelo.

In [ ]:
# calculate_ssr(intercept, abcissa (dataset), ordenada (dataset), slope)
def calculate_ssr(b, X, y, m):
    y_pred = m * X + b
    residuals = y - y_pred
    return np.sum(residuals**2)

Para uma melhor visualização gráfica, iremos armazenar a trajetória de b, do custo de b, e do gradiente a cada iteração, permitindo plotar a trajetória e a tangente na animação.

In [63]:
history_b = [b_start] # Armazena os valores de b ao longo das iterações
history_cost = [calculate_ssr(b_start, X, y, slope_fixed)] # Armazena os valores do custo ao longo das iterações
history_grads = [] # Armazena os valores do gradiente ao longo das iterações

current_b = b_start # Inicializa o valor atual de b

# Cálculo do gradiente inicial
y_pred_init = slope_fixed * X + current_b
grad_init = -2 * np.sum(y - y_pred_init)
history_grads.append(grad_init)

O laço de repetição abaixo, avança na quantidade definida (epochs), e calcula o gradiente, o novo valor de b e o custo no novo ponto, e armazena a trajetória.

In [64]:
for _ in range(epochs):
    # cálculo do gradiente no ponto atual
    y_pred = slope_fixed * X + current_b
    gradient = -2 * np.sum(y - y_pred)
    
    # atualiza b
    current_b = current_b - (learning_rate * gradient)
    
    # custo no novo ponto
    cost = calculate_ssr(current_b, X, y, slope_fixed)
    
    # recalcula gradiente no novo ponto para visualização correta
    y_pred_new = slope_fixed * X + current_b
    gradient_new = -2 * np.sum(y - y_pred_new)
    
    # armazena a trajetória
    history_b.append(current_b)
    history_cost.append(cost)
    history_grads.append(gradient_new)

Abaixo configuramos as opções de exibição dos gráficos (tanto o da função de custo, quanto o do dataset).

In [ ]:
fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(12, 5))
plt.suptitle(f'Regressão Linear com Visualização da Derivada', fontsize=14)

b_range = np.linspace(-0.5, 2.5, 100)
cost_curve = [calculate_ssr(b_val, X, y, slope_fixed) for b_val in b_range]

ax_left.plot(b_range, cost_curve, 'b--', alpha=0.3, label='Curva de Custo')
point_left, = ax_left.plot([], [], 'ro', zorder=5, label='Ponto Atual')
line_track, = ax_left.plot([], [], 'r-', alpha=0.3)


tangent_line, = ax_left.plot([], [], 'g-', linewidth=2, label='Derivada (Tangente)')

ax_left.set_title("Minimização do Custo (J)")
ax_left.set_xlabel("Intercepto (b)")
ax_left.set_ylabel("Erro (SSR)")
ax_left.legend(loc='upper center')
ax_left.grid(True, linestyle=':', alpha=0.6)
ax_left.set_ylim(0, max(cost_curve))

<IPython.core.display.Javascript object>

(0.0, 10.123999999999999)

In [ ]:
ax_right.scatter(X, y, color='blue', s=100)
line_right, = ax_right.plot([], [], 'r-', linewidth=2)
ax_right.set_xlim(0, 5)
ax_right.set_ylim(0, 5)
ax_right.set_title("Ajuste da Linha")
ax_right.grid(True, linestyle=':', alpha=0.6)

Definimos as funções necessárias para plotar de forma animada no gráfico, criando a função que define os valores iniciais, e a função que atualiza os valores a cada iteração.

In [ ]:
def init():
    point_left.set_data([], [])
    line_track.set_data([], [])
    tangent_line.set_data([], []) # Limpa tangente
    line_right.set_data([], [])
    return point_left, line_track, tangent_line, line_right

def update(frame):
    b = history_b[frame]
    cost = history_cost[frame]
    grad = history_grads[frame] # Pega o gradiente do frame atual
    
    # -- Atualiza Esquerda --
    point_left.set_data([b], [cost])
    line_track.set_data(history_b[:frame+1], history_cost[:frame+1])
    
    # Lógica da Reta Tangente: y = y0 + m(x - x0)
    # Criamos um pequeno intervalo de x ao redor do ponto b atual para desenhar a linha
    x_tan = np.linspace(b - 0.4, b + 0.4, 10) 
    y_tan = cost + grad * (x_tan - b)
    
    tangent_line.set_data(x_tan, y_tan)
    
    # -- Atualiza Direita --
    x_line = np.array([0, 5])
    y_line = slope_fixed * x_line + b
    line_right.set_data(x_line, y_line)
    
    return point_left, line_track, tangent_line, line_right


anim = FuncAnimation(fig, update, frames=len(history_b), init_func=init, interval=200, blit=True)
anim_html = anim.to_jshtml()
HTML(anim_html)

# Questão B

# Questão C

# Questão D

# Questão E